# Урок 6. Сравнение долей. Построение доверительного интервала

In [1]:
from math import factorial

import numpy as np
import pandas as pd

import scipy.stats as stats

***
## Задание 1.

### На препарате А положительный результат лечения наблюдается у 17 из 32 пациентов, а на препарате В у 9 из 22. Построить 95% доверительный интервал для разности долей. Обнаружены ли статистически значимые различия?

$H_0: p_1 = p_2$ \
$H_1: p_1 > p_2$ - правосторонняя критическая область

In [2]:
n1, m1, n2, m2, gamma = 32, 17, 22, 9, 0.95
alpha = 1 - gamma

p1, p2 = m1/n1, m2/n2
print(f'{p1 = :.2f}, {p2 = :.2f}')

p = (m1+m2)/(n1+n2)
print(f'{p = :.2f}')

Sd = np.sqrt(p*(1-p)*(1/n1+1/n2))

Zk = stats.norm.ppf(1-alpha/2)
print(f'{Zk = :.2f}')

delta = p1-p2
print(f'{delta-Zk*Sd:.2f} < {delta:.2f} < {delta+Zk*Sd:.2f}')

p1 = 0.53, p2 = 0.41
p = 0.48
Zk = 1.96
-0.15 < 0.12 < 0.39


Т.к. интервал захватывает 0, то статистически значимых различий не обнаружено. Т.е. препараты имеют одинаковую эффективность.

In [3]:
n1, m1, n2, m2, gamma = 32, 17, 22, 9, 0.95
alpha = 1 - gamma

Zk = stats.norm.ppf(1-alpha/2)
print(f'{Zk = :.2f}')

Zn = (m1/n1-m2/n2)/np.sqrt((m1+m2)/(n1+n2) * (1 - (m1+m2)/(n1+n2)) * (1/n1+1/n2)) 
print(f'{Zn = :.2f}')

print(f'H0: p1 = p2 is {Zn<Zk}')

Zk = 1.96
Zn = 0.88
H0: p1 = p2 is True


***
## Задание 2.

### Было проведено исследование научных статей на количество авторов в разные годы. Построить 90% и 95% интервалы.

| Год | Число статей | Среднее число авторов | Стандартное отклонение |
|-----|-----|-----|-----|
| 1946 | 151 | 2 | 1.4 |
| 1956 | 149 | 2.3 | 1.6 |
| 1966 | 157 | 2.8 | 1.2 |
| 1976 | 155 | 4.9 | 7.3 |

In [4]:
df = pd.DataFrame({'god': [1946, 1956, 1966, 1976],
                   'N': [151, 149, 157, 155],
                   'Mx': [2, 2.3, 2.8, 4.9],
                   'SD': [1.4, 1.6, 1.2, 7.3]})
df

,god,N,Mx,SD
0,1946,151,2.0,1.4
1,1956,149,2.3,1.6
2,1966,157,2.8,1.2
3,1976,155,4.9,7.3


In [5]:
def get_interval(row):
    l = round(row[2] - z * row[3] / np.sqrt(row[1]),2)
    r = round(row[2] + z * row[3] / np.sqrt(row[1]),2)
    return pd.Interval(left=l, right=r)

In [6]:
alpha = 0.1
z = stats.norm.ppf(1-alpha/2)
z

1.6448536269514722

In [7]:
df['int_90'] = df.apply(get_interval, axis=1)
df

,god,N,Mx,SD,int_90
0,1946,151,2.0,1.4,"(1.81, 2.19]"
1,1956,149,2.3,1.6,"(2.08, 2.52]"
2,1966,157,2.8,1.2,"(2.64, 2.96]"
3,1976,155,4.9,7.3,"(3.94, 5.86]"


In [8]:
alpha = 0.05
z = stats.norm.ppf(1-alpha/2)
z

1.959963984540054

In [9]:
df['int_95'] = df.apply(get_interval, axis=1)
df

,god,N,Mx,SD,int_90,int_95
0,1946,151,2.0,1.4,"(1.81, 2.19]","(1.78, 2.22]"
1,1956,149,2.3,1.6,"(2.08, 2.52]","(2.04, 2.56]"
2,1966,157,2.8,1.2,"(2.64, 2.96]","(2.61, 2.99]"
3,1976,155,4.9,7.3,"(3.94, 5.86]","(3.75, 6.05]"


***
## Задание 3.

### С помощью 90% доверительного интервала оценить средний вес нормально распределенной популяции, если дисперсия генеральной совокупности 3.6, а среднее арифметичекое по выборке объемом 100 получилось равным 71.2.

In [10]:
gamma, D, N, Mx = 0.9, 3.6, 100, 71.2
alpha = 1 - gamma

z = stats.norm.ppf(1-alpha/2)
z

1.6448536269514722

In [11]:
se = np.sqrt(D/N)
se

0.18973665961010278

In [12]:
print(f'{Mx-z*se:.2f} < a < {Mx+z*se:.2f}')

70.89 < a < 71.51


***
## Задание 4.

### Найдите 95% доверительные интервалы для долей больных, которые не чувствовали боли при включенном и выключенном приборе. Можно ли по этим интервалам оценить статистическую значимость различий?

|  | Прибор включен | Прибор выключен |
| ----- | ----- | ------ |
| Боли нет | 24 | 3 |
| Боль есть | 6 | 17 |

Выборочная оценка по доли имеет стандартную ошибку: \
$se = \sqrt{\cfrac{p(1-p)}{n}}$

По формуле Бернулли (биноминальное рапределение): \
$P_n(X=k) = C^k_n\cdot p^k\cdot q^{n-k}$, где \
$k$ - число наступления события (дискретная величина из отрезка $[0,n]$)<br>
$n$ - число испытаний<br>
$p$ - вероятность наступления события $A$ в независимых испытаниях<br>
$q = 1 - p$

In [13]:
n1, m1, n2, m2 = 30, 24, 20, 3
p1, p2 = m1/n1, m2/n2
alpha = 0.05

z = stats.norm.ppf(1-alpha/2)
z

1.959963984540054

In [14]:
se = np.sqrt(p1 * (1-p1) / n1)
se

0.07302967433402215

In [15]:
print(f'{p1-z*se:.2f} < p1 < {p1+z*se:.2f}')

0.66 < p1 < 0.94


In [16]:
# сочетание C - кол-во способов взять k элем-ов из множ-ва n без учета порядка расположения
def C(n, k):
    return factorial(n) // (factorial(k) * (factorial(n - k)))

In [17]:
l = C(n2,0) * p2**0 * (1-p2)**(n2-0)
gamma = l
for k in range(1,n2+1):
    p = C(n2,k) * p2**k * (1-p2)**(n2-k)
    gamma += p
    if gamma > 0.95: break
    print(f'{k = }, {p = }, {gamma = }')

print(f'{l:.2f} < p2 < {(k-1)/n2}')

k = 1, p = 0.13679834500416826, gamma = 0.1755578760886826
k = 2, p = 0.22933840191875263, gamma = 0.4048962780074352
k = 3, p = 0.24282889614926745, gamma = 0.6477251741567027
k = 4, p = 0.18212167211195063, gamma = 0.8298468462686533
k = 5, p = 0.10284517954557212, gamma = 0.9326920258142255
0.04 < p2 < 0.25


Т.к. интервалы не пересекаются, то можно сделать вывод, что есть статистически значимые различия.